# Using GraphRAG-SDK to Create a Multi-Agent-Orchestrator RAG System based on Knowledge Graph from Structured Data

GraphRAG-SDK provides a powerful tool, enhanced by LLM technology, for building a Multi-Agent-Orchestrator RAG System. This example demonstrates how to create a knowledge graph-based multi-agent system from structured data and how to use the orchestrator to manage the agents and perform question answering.

In [ ]:
!pip install graphrag_sdk[litellm]

In [ ]:
from json import loads
from dotenv import load_dotenv
from graphrag_sdk.orchestrator import Orchestrator
from graphrag_sdk.agents.kg_agent import KGAgent
from graphrag_sdk.models.litellm import LiteModel
from graphrag_sdk import (
    Ontology, Entity, Relation, Attribute, AttributeType, KnowledgeGraph, KnowledgeGraphModelConfig
)

# Load environment variables
load_dotenv()

# Configuration
# Set your OpenAI credential in .env file
# OPENAI_API_KEY=your_api_key

/home/shubeli/miniconda3/envs/sdk/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


True

### Ontology - Predefined Ontology

We will begin with an initial ontology for the agents, based on the knowledge that interests us.


In [2]:
# Restaurants Ontology
restaurants_ontology = Ontology()

# Attractions Ontology
attractions_ontology = Ontology()

# Manually created Ontology by adding entities and relations
restaurants_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)

attractions_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)

### KG from Structure Data

Creating the KG from the data that will be used by the agents

In [ ]:
# Define the model
# Using OpenAI with gpt-4.1 as default (temperature=0 by default)
model = LiteModel()

# Create the KG from the predefined ontology.
# Restaurants KG
restaurants_kg = KnowledgeGraph(
    name="restaurants",
    ontology=restaurants_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Attractions KG
attractions_kg = KnowledgeGraph(
    name="attractions",
    ontology=attractions_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Add nodes and edges to the KGs from the data
with open("examples/trip/data/cities.json") as f:
    cities = loads(f.read())
with open("examples/trip/data/restaurants.json") as f:
    restaurants = loads(f.read())
with open("examples/trip/data/attractions.json") as f:
    attractions = loads(f.read())

for city in cities:
    restaurants_kg.add_node(
        "City",
        {
            "name": city["name"],
            "weather": city["weather"],
            "population": city["population"],
        },
    )
    restaurants_kg.add_node("Country", {"name": city["country"]})
    restaurants_kg.add_edge(
        "IN_COUNTRY",
        "City",
        "Country",
        {"name": city["name"]},
        {"name": city["country"]},
    )

    attractions_kg.add_node(
        "City",
        {
            "name": city["name"],
            "weather": city["weather"],
            "population": city["population"],
        },
    )
    attractions_kg.add_node("Country", {"name": city["country"]})
    attractions_kg.add_edge(
        "IN_COUNTRY",
        "City",
        "Country",
        {"name": city["name"]},
        {"name": city["country"]},
    )

for restaurant in restaurants:
    restaurants_kg.add_node(
        "Restaurant",
        {
            "name": restaurant["name"],
            "description": restaurant["description"],
            "rating": restaurant["rating"],
            "food_type": restaurant["food_type"],
        },
    )
    restaurants_kg.add_edge(
        "IN_CITY",
        "Restaurant",
        "City",
        {"name": restaurant["name"]},
        {"name": restaurant["city"]},
    )

for attraction in attractions:
    attractions_kg.add_node(
        "Attraction",
        {
            "name": attraction["name"],
            "description": attraction["description"],
            "type": attraction["type"],
        },
    )
    attractions_kg.add_edge(
        "IN_CITY",
        "Attraction",
        "City",
        {"name": attraction["name"]},
        {"name": attraction["city"]},
    )

### Agents

Creating the Agents based on the KG.

In [6]:
# The following agent is specialized in finding restaurants.
restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="I'm a restaurant agent, specialized in finding the best restaurants for you.",
)

# The following agent is specialized in finding tourist attractions.
attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="I'm an attractions agent, specialized in finding the best attractions for you.",
)

### Orchestrator - Multi-Agent System

The orchestrator manages the usage of agents and handles questioning. Each agent is registered with the orchestrator, which determines how to engage the agents to provide the best possible answer when a question is asked.

In [7]:
model_orchestrator = LiteModel("o3-mini")

# Initialize the orchestrator and register agents
orchestrator = Orchestrator(
    model_orchestrator,
    backstory="You are a trip planner, and you want to provide the best possible itinerary for your clients.",
)
orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

# Query the orchestrator
runner = orchestrator.ask("Create a two-day itinerary for a trip to Rome. Please don't ask me any questions. Just provide the best itinerary you can.")

In [8]:
print(runner.output)

Dear Traveler,

Here is your detailed two-day itinerary for a memorable trip to Rome, designed to help you experience the city's rich history, culture, and culinary delights. I’ve combined the top attractions with excellent dining recommendations to ensure you have a well-rounded experience.

────────────────────────────
Day 1:
────────────────────────────
• Morning – Begin your day with a visit to the Colosseum. This iconic ancient amphitheater not only offers a glimpse into the grandeur of ancient Rome but also provides a stirring reminder of the gladiatorial contests and public spectacles that once took place there.

• Afternoon – Head over to the Vatican Museums. This renowned collection of art and historical artifacts, accumulated by Popes over centuries, is a must-see. The museums offer fascinating insights into Renaissance art and history, making it a perfect cultural stop.

• Lunch Suggestion – Enjoy lunch at Trattoria da Enzo. This well-loved eatery is known for its traditiona

In [9]:
runner = orchestrator.ask("Please tell me only the name of the restaurant for dinner at the first day that you mention in the itinerary of the trip")
print(runner.output)

Dear Traveler,

Thank you for your inquiry. Based on the itinerary details provided, the recommended restaurant for dinner on Day 1 is "Il Pagliaccio." This Michelin-starred restaurant was selected for its elegant ambiance and contemporary Italian cuisine, ensuring a refined dining experience that complements the rich cultural exploration of your trip.

I hope this clear and concise response helps you in planning your dining experience. Enjoy your journey and the delightful culinary offerings of Rome!

Best regards,
Your Trip Planning Team


In [10]:
runner = orchestrator.ask("Can you change this restaurant to another one and give me the updated itinerary?")
print(runner.output)

Dear Traveler,

Thank you for your request. I have updated your itinerary by replacing the original Day 1 dinner restaurant with an alternative option. Please find your updated itinerary below:

────────────────────────────
Day 1:
────────────────────────────
• Lunch – Enjoy a delightful meal at Trattoria da Enzo, a cozy trattoria celebrated for its traditional Roman dishes and warm atmosphere.

• Dinner – For dinner, you'll now have the chance to experience Tonnarello, a charming restaurant offering a casual setting with delicious pasta, meatballs, and other classic Roman dishes. This venue features patio seating accompanied by acoustic guitar, providing a relaxed yet engaging dining experience.

────────────────────────────
Day 2:
────────────────────────────
• Lunch – Once again, Tonnarello offers a satisfying meal with its array of authentic Roman dishes, ideal for a casual stop.

• Dinner – Conclude your Day 2 dining experience at Trattoria da Enzo, which ensures you continue to e

In [11]:
runner = orchestrator.ask("Please tell me the attraction on the morning at the first day that you mention in the itinerary of the trip")
print(runner.output)

Dear Traveler,

Thank you for your inquiry. Based on the itinerary details, the attraction scheduled for the morning of Day 1 is the Colosseum. The Colosseum, an iconic monument of ancient Rome, was chosen as it provides a firsthand look into the rich history and architectural prowess of the Roman Empire, setting a perfect tone for your day of exploration.

I hope this detailed explanation helps you understand your itinerary better. Enjoy your visit to this magnificent historical site and have a wonderful trip!

Best regards,
Your Trip Planning Team
